In [ ]:
RR = Perf[Perf['Year_and_Month']>'2016-12-01T00:00:00.000000000']
# get the list of year_and_month, 2016-12 to 2019-10
ym = list(RR['Year_and_Month'].unique())
# list of group combo
# roll_group = ['000-000 to 001-030','001-030 to 031-060','031-060 to 061-090','061-090 to 091-120','091-120 to 121-150','121-150 to 151+']

##  Use `Delinquency_Bucket` as the DQ indicator
- If you want to use `Default_Status` as DQ indicator, simply replace `Delinquency_Bucket` to `Default_Status` in function rr_matrix() and rr_matrix_bal()

In [1]:
def rr(df, year_month_1,year_month_2):
    """
    Merge 2 snapshots together
    """
#     print("{} to {}".format(year_month_1,year_month_2))
    return df[df['Year_and_Month']==year_month_1].merge(df[df['Year_and_Month']==year_month_2],how = 'left', on = 'Unique_Order_ID')

### Roll Rate Matrix - 1 month, 3 month, ...

#### Unit Level

In [41]:
def rr_matrix(df,ym,period):
    l = []
    for i in range(len(ym)):
        if i + period > len(ym)-1:
            break
        else:
            dfs = rr(df,ym[i],ym[i+period])
            dfss = dfs.pivot_table(values = 'Unique_Order_ID',index = 'Delinquency_Bucket_y',columns = 'Delinquency_Bucket_x',aggfunc='count').rename_axis(None, axis=1)
            l.append(dfss)
    return l

In [29]:
matrix_1m = rr_matrix(RR,ym,1)

In [ ]:
# sum([matrix_1m[i] for i in range(24,30)])/6

In [42]:
matrix_3m = rr_matrix(RR,ym,3)

In [44]:
len(matrix_3m)

31

In [54]:
sum([matrix_3m[i] for i in range(24,30)])/6

,000-000,001-030,031-060,061-090,091-120,121-150,151+
Delinquency_Bucket_y,,,,,,,
000-000,417834.000000,26266.000000,1321.166667,892.666667,644.833333,712.166667,6577.000000
001-030,28726.000000,13449.833333,1042.166667,191.666667,47.333333,7.500000,NaN
031-060,11306.833333,3155.333333,1320.500000,320.166667,88.333333,19.333333,4.833333
061-090,8346.333333,2514.166667,985.500000,562.833333,139.000000,33.666667,20.166667
091-120,580.500000,8509.500000,1180.500000,567.000000,293.500000,67.000000,33.833333
121-150,18.166667,595.333333,8128.666667,832.500000,346.166667,194.166667,69.833333
151+,5.833333,10.166667,500.333333,8310.333333,8718.166667,8380.666667,14491.833333


#### $ Level

In [11]:
def rr_matrix_bal(df, ym,period):
    l = []
    for i in range(len(ym)):
        if i + period > len(ym)-1:
            break
        else:
            dfs = rr(df,ym[i],ym[i+period])
            dfss = dfs.pivot_table(values = 'Current_Balance_x',index = 'Delinquency_Bucket_y',columns = 'Delinquency_Bucket_x',aggfunc='sum').rename_axis(None, axis=1)
            l.append(dfss)
    
    return l

In [14]:
matrix_1m_bal = rr_matrix_bal(RR,ym,1)

In [21]:
sum([matrix_1m_bal[i] for i in range(6)])/6

,000-000,001-030,031-060,061-090,091-120,121-150,151+
Delinquency_Bucket_y,,,,,,,
000-000,1.838328e+08,6.608870e+06,2.033207e+05,6.431713e+04,4.615153e+04,7.881358e+04,1.692778e+06
001-030,1.197147e+07,1.008772e+07,4.637027e+05,3.592429e+04,8.533142e+03,1.566825e+03,NaN
031-060,1.978351e+05,4.020730e+06,1.534830e+06,1.718615e+05,1.121220e+04,3.545280e+03,NaN
061-090,1.044018e+04,6.741361e+04,3.202978e+06,7.258053e+05,8.114971e+04,9.474682e+03,NaN
091-120,2.678787e+03,2.426518e+03,6.979510e+04,2.748737e+06,3.725453e+05,4.467388e+04,5.770258e+03
121-150,9.801017e+02,NaN,1.754303e+03,5.244613e+04,2.429713e+06,2.472656e+05,3.523623e+04
151+,8.923617e+02,NaN,NaN,1.649922e+03,5.956865e+04,2.221859e+06,7.304536e+06


In [16]:
matrix_3m_bal = rr_matrix_bal(RR,ym,3)

In [22]:
sum([matrix_3m_bal[i] for i in range(24,30)])/6

,000-000,001-030,031-060,061-090,091-120,121-150,151+
Delinquency_Bucket_y,,,,,,,
000-000,1.980084e+08,1.154595e+07,4.833664e+05,3.051158e+05,2.571373e+05,3.359410e+05,2.472485e+06
001-030,1.458227e+07,6.823509e+06,5.515140e+05,1.030555e+05,2.256574e+04,3.434073e+03,NaN
031-060,5.354744e+06,1.559842e+06,7.526119e+05,1.823889e+05,4.809148e+04,1.114667e+04,2.819162e+03
061-090,3.686390e+06,1.156526e+06,5.201652e+05,3.058760e+05,7.900538e+04,1.880027e+04,9.578257e+03
091-120,2.657608e+05,3.657604e+06,5.857510e+05,3.133471e+05,1.605200e+05,3.616268e+04,1.645157e+04
121-150,1.073101e+04,2.733105e+05,3.491127e+06,3.994037e+05,1.739123e+05,8.989068e+04,3.195684e+04
151+,1.218273e+03,6.426295e+03,2.309793e+05,3.568203e+06,3.701976e+06,3.471941e+06,6.534511e+06
